In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Cargar los datos del CSV
data = pd.read_csv('exoPlanet.csv', delimiter=';')

# Función para calcular la SNR
def calculate_snr(R, RP, D, ES, PS, SNR0=100):
    return 2 * SNR0 * ((R * RP * (D / 6)) / ((ES / 10) * PS))

# Añadir una columna con el cálculo del SNR (asumiendo D = 6 metros para el telescopio HWO)
data['SNR'] = data.apply(lambda row: calculate_snr(row['Stellar Radius [Solar Radius]'],
                                                   row['Planet Radius [Earth Radius]'],
                                                   6,
                                                   row['Distance [pc]'],
                                                   row['Orbit Semi-Major Axis [au]']), axis=1)

# Normalizar el SNR en un rango de 0 a 100 para representar el porcentaje
data['visualization_percentage'] = data['SNR'].apply(lambda snr: min(100, max(0, (snr / 5) * 100)))

# Seleccionar las columnas relevantes para el modelo
features = ['Stellar Radius [Solar Radius]', 'Planet Radius [Earth Radius]',
            'Distance [pc]', 'Orbit Semi-Major Axis [au]']
X = data[features]
y = data['visualization_percentage']

# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Entrenar un modelo de regresión
model = RandomForestRegressor()
model.fit(X_train, y_train)

# Predecir en los datos de prueba
y_pred = model.predict(X_test)

# Calcular el error cuadrático medio
mse = mean_squared_error(y_test, y_pred)
print(f'Error cuadrático medio del modelo: {mse:.2f}')

# Probar con nuevos datos para predecir el porcentaje de visualización
def predict_visualization_percentage(R, RP, D, ES, PS):
    snr = calculate_snr(R, RP, D, ES, PS)
    percentage = min(100, max(0, (snr / 5) * 100))
    return percentage

# predicción
visualization_percentage = predict_visualization_percentage(1.1, 10, 6, 20, 1)
print(f'Porcentaje de visualización del planeta: {visualization_percentage:.2f}%')


Error cuadrático medio del modelo: 2.71
Porcentaje de visualización del planeta: 100.00%


In [ ]:
import pandas as pd
import numpy as np
import tkinter as tk
from tkinter import messagebox, simpledialog, Listbox, Scrollbar

# Cargar los datos del CSV
data = pd.read_csv('exoPlanet.csv', delimiter=';')

# Función para calcular la SNR
def calculate_snr(R, RP, D, ES, PS, SNR0=100):
    return 2 * SNR0 * ((R * RP * (D / 6)) / ((ES / 10) * PS))

# Función para predecir el porcentaje de visualización
def predict_visualization_percentage(R, RP, D, ES, PS):
    snr = calculate_snr(R, RP, D, ES, PS)
    percentage = min(100, max(0, (snr / 5) * 100))
    return percentage

# Función para calcular el diámetro necesario
def calculate_diameter(R, RP, ES, PS, SNR0=100):
    return (5 * (ES / 10) * PS) / (2 * SNR0 * R * RP) * 6

# Función que se ejecuta al presionar el botón para realizar la predicción
def on_predict():
    try:
        
        # Obtener los valores ingresados por el usuario
        R = float(entry_R.get() or 1.0)  # Valor predeterminado: 1.0 radios solares
        RP = float(entry_RP.get() or 1.0)  # Valor predeterminado: 1.0 radios terrestres
        D = 6  # Diámetro del telescopio es fijo en este caso
        ES = float(entry_ES.get() or 10)  # Valor predeterminado: 10 parsecs
        PS = float(entry_PS.get() or 1.0)  # Valor predeterminado: 1.0 UA

        # Verificar si hay valores NaN
        if None in (R, RP, ES, PS):
            messagebox.showerror("Datos Faltantes", "Faltan datos para la predicción. Por favor, complete todos los campos.")
            return

        # Realizar la predicción
        percentage = predict_visualization_percentage(R, RP, D, ES, PS)

        # Mostrar el resultado en la interfaz
        if percentage < 100:
            # Calcular el diámetro necesario
            required_diameter = calculate_diameter(R, RP, ES, PS)
            result_label.config(text=f"Porcentaje de visualización: {percentage:.2f}%\n"
                                      f"Diámetro del telescopio necesario para 100% es de: {required_diameter:.2f} m")
        else:
            result_label.config(text=f"Porcentaje de visualización: {percentage:.2f}%\n"
                                      f"El planeta es completamente observable.")

    except ValueError:
        messagebox.showerror("Entrada no válida", "Por favor, ingrese valores numéricos válidos.")

# Función para cargar los parámetros de un exoplaneta seleccionado
def load_exoplanet_parameters(event):
    selected_index = exoplanet_listbox.curselection()
    if selected_index:
        index = selected_index[0]
        planet = data.iloc[index]

        # Cargar los parámetros en los campos de entrada
        entry_R.delete(0, tk.END)
        entry_R.insert(0, planet['Stellar Radius [Solar Radius]'])
        
        entry_RP.delete(0, tk.END)
        entry_RP.insert(0, planet['Planet Radius [Earth Radius]'])
        
        entry_ES.delete(0, tk.END)
        entry_ES.insert(0, planet['Distance [pc]'])
        
        entry_PS.delete(0, tk.END)
        entry_PS.insert(0, planet['Orbit Semi-Major Axis [au]'])

# Crear la ventana principal de la interfaz
root = tk.Tk()
root.title("Predicción de Visualización de Planeta")

# Instrucción para el usuario
instructions = tk.Label(root, text="Ingrese los parámetros o use los valores predeterminados:", font=("Arial", 12))
instructions.grid(row=0, column=0, columnspan=2)

# Crear las etiquetas y campos de entrada para los parámetros
tk.Label(root, text="Radio Estelar [R] (predeterminado: 1.0)").grid(row=1, column=0)
entry_R = tk.Entry(root)
entry_R.grid(row=1, column=1)

tk.Label(root, text="Radio Planetario [RP] (predeterminado: 1.0)").grid(row=2, column=0)
entry_RP = tk.Entry(root)
entry_RP.grid(row=2, column=1)

tk.Label(root, text="Distancia al sistema [ES] (predeterminado: 10 parsecs)").grid(row=3, column=0)
entry_ES = tk.Entry(root)
entry_ES.grid(row=3, column=1)

tk.Label(root, text="Distancia planeta-estrella [PS] (predeterminado: 1 UA)").grid(row=4, column=0)
entry_PS = tk.Entry(root)
entry_PS.grid(row=4, column=1)

# Crear el botón para realizar la predicción
predict_button = tk.Button(root, text="Predecir", command=on_predict)
predict_button.grid(row=5, column=0, columnspan=2)

# Etiqueta para mostrar el resultado de la predicción
result_label = tk.Label(root, text="Porcentaje de visualización: ", font=("Arial", 12))
result_label.grid(row=6, column=0, columnspan=2)

# Crear la lista de exoplanetas
exoplanet_label = tk.Label(root, text="Seleccionar Exoplaneta:", font=("Arial", 12))
exoplanet_label.grid(row=7, column=0, columnspan=2)

# Crear un marco para la lista
frame = tk.Frame(root)
frame.grid(row=8, column=0, columnspan=2)

# Crear la lista de exoplanetas con un scrollbar
exoplanet_listbox = Listbox(frame, width=50, height=10)
exoplanet_listbox.pack(side=tk.LEFT, fill=tk.BOTH)

# Agregar scrollbar a la lista
scrollbar = Scrollbar(frame)
scrollbar.pack(side=tk.RIGHT, fill=tk.Y)
exoplanet_listbox.config(yscrollcommand=scrollbar.set)
scrollbar.config(command=exoplanet_listbox.yview)

# Agregar exoplanetas a la lista
for index, row in data.iterrows():
    exoplanet_listbox.insert(tk.END, f"{index + 1}: {row['Planet Name']} (Descubierto en {row['disc_year']})")

# Vincular el evento de selección de la lista
exoplanet_listbox.bind('<<ListboxSelect>>', load_exoplanet_parameters)

# Iniciar el bucle de la interfaz gráfica
root.mainloop()



In [29]:
import pandas as pd
data = pd.read_csv('exoPlanet.csv', delimiter=';')
print(data.columns)

Index(['disc_year', 'Planet Name', 'hostname', 'Number of Stars',
       'Number of Planets', 'Discovery Method', 'Orbital Period [days]',
       'Orbit Semi-Major Axis [au]', 'Planet Radius [Earth Radius]',
       'Planet Mass or Mass*sin(i) [Earth Mass]',
       'Planet Mass or Mass*sin(i) Provenance', 'Eccentricity',
       'Equilibrium Temperature [K]', 'Stellar Effective Temperature [K]',
       'Stellar Radius [Solar Radius]', 'Stellar Mass [Solar mass]',
       'Stellar Metallicity Ratio', 'Stellar Surface Gravity [log10(cm/s**2)]',
       'Distance [pc]'],
      dtype='object')
